In [1]:
import cudaq
from cudaq import spin
import networkx as nx
import numpy as np
from typing import Optional
import pandas as pd


In [2]:
number_of_qubits = 5
layer_count = 1
parameter_count = 4 * layer_count * number_of_qubits 

In [3]:
coreset_weights = np.random.random(number_of_qubits)
coreset_vectors = np.random.random((number_of_qubits,2))

In [4]:
def get_Hamil_variables(
    coreset_vectors: np.ndarray,
    coreset_weights: np.ndarray,
    index_vals_temp: Optional[int] = None,
    new_df: Optional[pd.DataFrame] = None,
):
    """
    Generates the variables required for Hamiltonian

    Args:
        coreset_vectors: Coreset vectors
        coreset_weights: Coreset weights
        index_vals_temp: Index in the hierarchy
        new_df: new dataframe create for this problem,

    Returns:
       Graph, weights and qubits
    """
    if new_df is not None and index_vals_temp is not None:
        coreset_weights, coreset_vectors = get_cv_cw(coreset_vectors, coreset_weights, index_vals_temp)
    
    coreset_points, G, H, weight_matrix, weights = gen_coreset_graph(
        coreset_vectors, coreset_weights, metric="dot"
    )
    qubits = len(G.nodes)

    return G, weights, qubits

def create_Hamiltonian_for_K2(G, qubits, weights: np.ndarray = None,add_identity=False):
    """
    Generate Hamiltonian for k=2

    Args:
        G: Problem as a graph
        weights: Edge weights
        nodes: nodes of the graph
        add_identity: Add identiy or not. Defaults to False.

    Returns:
        _type_: _description_
    """
    H = 0
#     if add_identity:
#         H = -1 * sum(weights) 
#     else:
#         H = 0 

    for i, j in G.edges():
        weight = G[i][j]["weight"]#[0]
        H += weight * (spin.z(i) * spin.z(j))
        
    return H
        
def gen_coreset_graph(
    coreset_vectors: np.ndarray,
    coreset_weights: np.ndarray,
    metric: str = "dot",
):
    """
    Generate a complete weighted graph using the provided set of coreset points

    Parameters
    ----------
    coreset_weights : ndarray
        np.Coreset weights in array format

    coreset_vectors : ndarray
        Data points of the coreset

    metric : str
        Choose the desired metric for computing the edge weights.
        Options include: dot, dist

    Returns
    -------
    coreset : List((weight, vector))
        The set of points used to construct the graph
    G : NetworkX Graph
        A complete weighted graph
    H : List((coef, pauli_string))
        The equivalent Hamiltonian for the generated graph
    weight_matrix : np.array
        Edge weights of the graph in matrix
    weights : np.array
        Edge weights of the graph in an array

    """

    coreset = [(w, v) for w, v in zip(coreset_weights, coreset_vectors)]

    if coreset is None:
        # Generate a graph instance with sample coreset data
        coreset = []
        # generate 3 points around x=-1, y=-1
        for _ in range(3):
            # use a uniformly random weight
            # weight = np.random.uniform(0.1,5.0,1)[0]
            weight = 1
            vector = np.array(
                [
                    np.random.normal(loc=-1, scale=0.5, size=1)[0],
                    np.random.normal(loc=-1, scale=0.5, size=1)[0],
                ]
            )
            new_point = (weight, vector)
            coreset.append(new_point)

        # generate 3 points around x=+1, y=1
        for _ in range(2):
            # use a uniformly random weight
            # weight = np.random.uniform(0.1,5.0,1)[0]
            weight = 1
            vector = np.array(
                [
                    np.random.normal(loc=1, scale=0.5, size=1)[0],
                    np.random.normal(loc=1, scale=0.5, size=1)[0],
                ]
            )
            new_point = (weight, vector)
            coreset.append(new_point)

    # Generate a networkx graph with correct edge weights
    n = len(coreset)
    G = nx.complete_graph(n)
    H = []
    weights = []
    weight_matrix = np.zeros(len(G.nodes) ** 2).reshape(len(G.nodes()), -1)
    for edge in G.edges():
        pauli_str = ["I"] * n
        # coreset points are labelled by their vertex index
        v_i = edge[0]
        v_j = edge[1]
        pauli_str[v_i] = "Z"
        pauli_str[v_j] = "Z"
        w_i = coreset[v_i][0]
        w_j = coreset[v_j][0]
        if metric == "dot":
            mval = np.dot(coreset[v_i][1], coreset[v_j][1])
        elif metric == "dist":
            mval = np.linalg.norm(coreset[v_i][1] - coreset[v_j][1])
        else:
            raise Exception("Unknown metric: {}".format(metric))

        weight_val = w_i * w_j
        weight_matrix[v_i, v_j] = weight_val
        weight_matrix[v_j, v_i] = weight_val
        G[v_i][v_j]["weight"] = w_i * w_j * mval
        weights.append(w_i * w_j * mval)
        H.append((w_i * w_j * mval, pauli_str))

    return coreset, G, H, weight_matrix, weights

In [5]:
G, weights, qubits = get_Hamil_variables(coreset_vectors, coreset_weights)
H = create_Hamiltonian_for_K2(G=G,qubits=qubits )

In [6]:
def kernel_two_local() -> cudaq.Kernel:
    """QAOA ansatz for maxcut"""
    kernel, thetas = cudaq.make_kernel(list)
    qreg = kernel.qalloc(number_of_qubits)

    # Create superposition
    kernel.h(qreg)

    # Loop over the layers
    theta_position = 0
    
    for i in range(layer_count):
        # Loop over the qubits
        # Problem unitary
        for j in range(number_of_qubits):
            kernel.rz(thetas[theta_position], qreg[j % number_of_qubits])
            kernel.rx(thetas[theta_position + 1], qreg[j % number_of_qubits])
            kernel.cx(qreg[j], qreg[(j + 1) % number_of_qubits])
            kernel.rz(thetas[theta_position + 2], qreg[j % number_of_qubits])
            kernel.rx(thetas[theta_position + 3], qreg[j % number_of_qubits])
            theta_position += 4



    return kernel

In [7]:
optimizer = cudaq.optimizers.COBYLA()
optimizer.initial_parameters = np.random.uniform(-np.pi / 8.0, np.pi / 8.0,
                                                 parameter_count)
print("Initial parameters = ", optimizer.initial_parameters)

Initial parameters =  [0.30519878221702423, 0.25554793654487595, -0.06790056565932406, -0.12636855695090732, -0.1831258025061872, 0.24601812168900483, 0.3510832468812629, 0.2859486701086392, -0.24408330682974275, 0.22699750294697285, -0.00048167699892559535, -0.3853229847823712, -0.1750010600336003, -0.03561673270141558, -0.09251556488143253, -0.32870087731583286, 0.0620622243403417, -0.33955255109516025, 0.2938222289502779, -0.3517157719854222]


In [8]:
optimal_expectation, optimal_parameters = cudaq.vqe(
    kernel=kernel_two_local(),
    spin_operator=H,
    optimizer=optimizer,
    parameter_count=parameter_count,
    shots = 100)

# Print the optimized value and its parameters
print("Optimal value = ", optimal_expectation)
print("Optimal parameters = ", optimal_parameters)

# Sample the circuit using the optimized parameters
counts = cudaq.sample(kernel_two_local(), optimal_parameters, shots_count = 100)


<H> = -0.008183
<H> = -0.024852
<H> = 0.121700
<H> = 0.045980
<H> = 0.124689
<H> = -0.010625
<H> = -0.026821
<H> = -0.022530
<H> = -0.013096
<H> = -0.027071
<H> = -0.025616
<H> = -0.026725
<H> = -0.025617
<H> = -0.010194
<H> = -0.018371
<H> = -0.011700
<H> = -0.013924
<H> = 0.048423
<H> = -0.033311
<H> = 0.017396
<H> = 0.121220
<H> = -0.061739
<H> = -0.079753
<H> = -0.085806
<H> = -0.103392
<H> = -0.063657
<H> = -0.099765
<H> = -0.081532
<H> = -0.117722
<H> = -0.134178
<H> = -0.142427
<H> = -0.145937
<H> = -0.144825
<H> = -0.155216
<H> = -0.144121
<H> = -0.188635
<H> = -0.215491
<H> = -0.248483
<H> = -0.267979
<H> = -0.031413
<H> = -0.264379
<H> = -0.251909
<H> = -0.257180
<H> = -0.283941
<H> = -0.301159
<H> = -0.280790
<H> = -0.371924
<H> = -0.379419
<H> = -0.397265
<H> = -0.370463
<H> = -0.402497
<H> = -0.373944
<H> = -0.423052
<H> = -0.430905
<H> = -0.424521
<H> = -0.420799
<H> = -0.407173
<H> = -0.492824
<H> = -0.389765
<H> = -0.457969
<H> = -0.475069
<H> = -0.485264
<H> = -0.38939

<H> = -0.680617
<H> = -0.680609
<H> = -0.680601
<H> = -0.680614
<H> = -0.680625
<H> = -0.680623
<H> = -0.680597
<H> = -0.680626
<H> = -0.680620
<H> = -0.680627
<H> = -0.680609
<H> = -0.680630
<H> = -0.680597
<H> = -0.680631
<H> = -0.680608
<H> = -0.680629
<H> = -0.680611
<H> = -0.680629
<H> = -0.680626
<H> = -0.680628
<H> = -0.680591
<H> = -0.680631
<H> = -0.680622
<H> = -0.680631
<H> = -0.680608
<H> = -0.680629
<H> = -0.680627
<H> = -0.680631
<H> = -0.680595
<H> = -0.680632
<H> = -0.680619
<H> = -0.680631
<H> = -0.680618
<H> = -0.680631
<H> = -0.680614
<H> = -0.680610
<H> = -0.680626
<H> = -0.680632
<H> = -0.680629
<H> = -0.680632
<H> = -0.680627
<H> = -0.680632
<H> = -0.680625
<H> = -0.680634
<H> = -0.680631
<H> = -0.680633
<H> = -0.680635
<H> = -0.680637
<H> = -0.680636
<H> = -0.680635
<H> = -0.680635
<H> = -0.680637
<H> = -0.680630
<H> = -0.680637
<H> = -0.680634
<H> = -0.680639
<H> = -0.680631
<H> = -0.680637
<H> = -0.680634
<H> = -0.680637
<H> = -0.680638
<H> = -0.680639
<H> = -0

In [9]:
print(counts)

{ 01100:100 }



In [11]:
import cudaq
from cudaq import spin

import numpy as np

# Here we build up a kernel for QAOA with p layers, with each layer
# containing the alternating set of unitaries corresponding to the problem
# and the mixer Hamiltonians. The algorithm leverages the VQE algorithm
# to compute the maxcut of a rectangular graph illustrated below.

#       v0  0---------------------0 v1
#           |                     |
#           |                     |
#           |                     |
#           |                     |
#       v3  0---------------------0 v2
# The maxcut for this problem is 0101 or 1010.

# The problem Hamiltonian
hamiltonian = 0.5 * spin.z(0) * spin.z(1) + 0.5 * spin.z(1) * spin.z(2) \
       + 0.5 * spin.z(0) * spin.z(3) + 0.5 * spin.z(2) * spin.z(3)

# Problem parameters.
qubit_count: int = 4
layer_count: int = 2
parameter_count: int = 2 * layer_count


def kernel_qaoa() -> cudaq.Kernel:
    """QAOA ansatz for maxcut"""
    kernel, thetas = cudaq.make_kernel(list)
    qreg = kernel.qalloc(qubit_count)

    # Create superposition
    kernel.h(qreg)

    # Loop over the layers
    for i in range(layer_count):
        # Loop over the qubits
        # Problem unitary
        for j in range(qubit_count):
            kernel.cx(qreg[j], qreg[(j + 1) % qubit_count])
            kernel.rz(2.0 * thetas[i], qreg[(j + 1) % qubit_count])
            kernel.cx(qreg[j], qreg[(j + 1) % qubit_count])

        # Mixer unitary
        for j in range(qubit_count):
            kernel.rx(2.0 * thetas[i + layer_count], qreg[j])

    return kernel


# Specify the optimizer and its initial parameters.
optimizer = cudaq.optimizers.COBYLA()
optimizer.initial_parameters = np.random.uniform(-np.pi / 8.0, np.pi / 8.0,
                                                 parameter_count)
print("Initial parameters = ", optimizer.initial_parameters)

# Pass the kernel, spin operator, and optimizer to `cudaq.vqe`.
optimal_expectation, optimal_parameters = cudaq.vqe(
    kernel=kernel_qaoa(),
    spin_operator=hamiltonian,
    optimizer=optimizer,
    parameter_count=parameter_count)

# Print the optimized value and its parameters
print("Optimal value = ", optimal_expectation)
print("Optimal parameters = ", optimal_parameters)

# Sample the circuit using the optimized parameters
counts = cudaq.sample(kernel_qaoa(), optimal_parameters)
counts.dump()

Initial parameters =  [0.014905453806909885, 0.23364382198795952, 0.05952651217294852, -0.051798953601863984]
Optimal value =  -1.9999997515935108
Optimal parameters =  [-1.1185607766208203, -1.011423337959866, -0.5592384393769969, -0.4521765071652822]


In [12]:
print(counts)

{ 1010:523 0101:477 }



In [34]:
[i for i in counts.items()]

[('1010', 523), ('0101', 477)]

In [33]:
counts.items()